**Setup**

In [ ]:
# 导入依赖

import re
import torch
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import vllm.model_executor.layers.logits_processor as LPmod
from vllm.model_executor.sampling_metadata import SamplingMetadata
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import snapshot_download, hf_hub_download
from collections import defaultdict
from datasets import load_dataset
from rich.console import Console
from rich.text import Text

# 设置环境变量

import os
import json
from IPython.display import clear_output

os.environ['RANK'] = '0'
os.environ['VLLM_USE_V1'] = '0'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 设置常量

ADD_GENERATION_PROMPT = True

# 导入配置

config = json.load(open('../config.json', 'r'))

# 定义函数
import pandas as pd
import numpy as np
from pandas import Series

# 统计函数：四分位距
def iqr(x: list[float]):
    return np.percentile(x, 75) - np.percentile(x, 25)

def select_next(stats: Series):
    """
    从统计结果中筛选出：
    - 数据长度最长的组 (count 最大)
    - 平均值最高的组 (mean 最大)
    - 集中度最高的组 (iqr 最小)
    返回这些组索引组成的集合
    """
    idx_len = stats["count"].idxmax()        # 数据最长
    idx_mean = stats["mean"].idxmax()        # 平均值最高
    idx_concentration = stats["iqr"].idxmin()  # 集中度最高（std 最小）

    return {idx_len, idx_mean, idx_concentration}

# 导入数据集
dataset_path = config['dataset']
result_path = config['result']

with open(dataset_path, 'r') as f:
    lines = f.readlines()
    questions: list[str] = [json.loads(line)['question'] for line in lines][0:4]
    answers: list[str] = [json.loads(line)['answer'] for line in lines][0:4]

final_answers = [int(answer.split('#### ')[-1]) for answer in answers]
dataset = list(zip(questions, final_answers))


# 加载目标模型
from vllm import LLM, SamplingParams, RequestOutput

target_model = LLM(
    model=config['models'][config['model']],
    task="generate",
    enable_chunked_prefill=False,
    max_model_len=config['max_model_len'],
    gpu_memory_utilization=config['gpu_memory_utilization']
)

target_tokenizer = target_model.get_tokenizer()

# 加载奖励模型

reward_model = LLM(
    model="/root/project/models/Qwen2.5-Math-7B-PRM800K",
    task="reward",
    max_model_len=8192,
    enable_chunked_prefill=False,
    gpu_memory_utilization=0.25
)

reward_tokenizer = reward_model.get_tokenizer()

# 设置采样参数
# 第一阶段采样

generating_params_1 = SamplingParams(
    temperature=0.7,
    max_tokens=1024,
    stop = ["\n\n"],
    n=10
)

# 第二阶段采样

generating_params_2 = SamplingParams(
    temperature=0.7,
    max_tokens=8192
)

0. 奖励分数统计优化算法

In [ ]:
for question, answer in dataset:
    print(f'Question: {question}')

    input_text = question + f"{str(answer)} is wrong." # 改写问题
    inputs = [target_tokenizer.apply_chat_template(
        [
            {
                'role':'user',
                'content':content
            }
        ],
        tokenize=False,
        add_generation_prompt=ADD_GENERATION_PROMPT
    ) for content in input_text]

    print(f"Counterfactual: {input_text}")

    # 迭代优化
    # 1. 采样模型回答
    # 2. 统计奖励得分
    # 3. 选择下一轮输入


    gen_of_thinking = 0
    flag = True
    while(flag and gen_of_thinking<50): # 终止条件：产生循环思考或超过一定轮次

        print(f'Generation: {gen_of_thinking}')

        # 采样目标模型回答
        outputs: list[RequestOutput] = target_model.generate(inputs, generating_params_2)

        # 用奖励模型为一批样本的每步思考打分
        thinkings: list[str] = []
        prompts: list[str] = []
        thinking_length: list[int] = []

        for i, output in enumerate(outputs):
            print(f"Input: {input_text[i]}")
            for thinking in output.outputs:
                text = thinking.text
                thinking_length.append(len(thinking.token_ids))
                thinkings.append(text)
                steps = text.split('\n\n')

                # 若有输出达到最大模型长度，且重复思考步骤，则终止优化，记录输出
                total_token_num = len(target_tokenizer.encode(inputs[i])) + len(thinking.token_ids)
                print(f"token 总数: {total_token_num}")
                if total_token_num >= 4096:
                    print(f"Last 10 steps of a long thinking:\n\n{'\n\n'.join(steps[-10:])}")
                    if text.count(steps[-2]) > 1:
                        flag = False

                messages = [
                    {"role": "user", "content": input_text[i]},
                    {"role": "assistant", "content": "<extra_0>".join(steps) + "<extra_0>"},
                ]

                # 输入截断
                token_ids = reward_tokenizer.encode(
                    reward_tokenizer.apply_chat_template(
                        messages, 
                        tokenize=False
                    ),
                    truncation=True,
                    max_length=4096,
                    add_special_tokens=False
                )

                prompt = reward_tokenizer.decode(token_ids)
                pos = prompt.rfind("<extra_0>")
                prompts.append(prompt[:pos+9])

        rewards = reward_model.encode(prompts)

        # 对奖励得分结果进行统计
        scores = [reward.outputs.data.cpu().numpy()[:,1] for reward in rewards]
        df = pd.DataFrame([
            {"thinking": i, "reward": v}
            for i, arr in enumerate(scores)
            for v in arr
        ])

        # 选择进行迭代的样本
        # 筛选标准：
        # 1. 思考长度
        # 2. 平均得分
        # 3. 得分集中度
        stats = df.groupby("thinking")["reward"].agg(mean="mean")
        stats["count"] = thinking_length
        stats["iqr"] = df.groupby("thinking")["reward"].apply(iqr)

        next_generations = select_next(stats)

        # 获取箱型图中得分异常思考步的索引
        first_outlier_indices = []

        for generation in next_generations:
            data = Series(scores[generation])
            Q1 = data.quantile(0.25)
            IQR = iqr(scores[generation])
            lower_bound = Q1 - 1.5 * IQR
            
            # 找出异常值的布尔掩码
            outliers_mask = (data < lower_bound)

            # 获取异常值的索引
            outlier_indices = data.index[outliers_mask].tolist()

            # 第一个异常值索引
            if len(outlier_indices) > 1:
                first_outlier_indices.append(outlier_indices[1])
            else:
                first_outlier_indices.append(data.idxmin())

        inputs = [
            target_tokenizer.apply_chat_template(
                [
                    {
                        'role':'user',
                        'content':'\n\n'.join(thinkings[generation].split('\n\n')[:idx+1])
                    }
                ],
                tokenize=False,
                add_generation_prompt=ADD_GENERATION_PROMPT
            ) for generation, idx in zip(next_generations, first_outlier_indices)
        ]

        input_text = ['\n\n'.join(thinkings[generation].split('\n\n')[:idx+1]) for generation, idx in zip(next_generations, first_outlier_indices)]

        gen_of_thinking += 1

    break

1. 奖励分数贪婪搜索引导算法

In [ ]:
input_str = target_tokenizer.apply_chat_template(
    context,
    add_generation_prompt=True,
    tokenize=False
)

best_steps = []
best_steps_with_score = []
i = 0

while(i < 50):

    # 采样一步思考
    output: list[RequestOutput] = target_model.generate(
        input_str,
        sampling_params=generating_params_1
    )

    thinking_steps = [item.text + "\n\n" for item in output[0].outputs]
    context_with_rank_token = [input_str + thinking+ "<extra_0>" for thinking in thinking_steps]

    rewards = reward_model.encode(context_with_rank_token)

    scores = [reward.outputs.data.cpu().tolist()[0][1] for reward in rewards]

    choosed_step = thinking_steps[scores.index(max(scores))]

    best_steps_with_score.append(choosed_step[:-1] + f"[{str(max(scores))}]" + "\n")

    input_str += choosed_step

    clear_output(wait=True)
    print(f"User: {context[0]['content']}\n\nAssistant: {''.join(best_steps_with_score)}")

    if output[0].outputs[scores.index(max(scores))].stop_reason == None:# or choosed_step.endswith("</think>"):
        break

    i += 1

2. 多阶段奖励分数束搜索引导算法

In [ ]:
# 第一阶段采样

output: list[RequestOutput] = target_model.generate(
    input_str,
    generating_params_1
)

# 第二阶段采样

i = 0
k = 3
s = 10
flag = True

steps = answers[1].split("\n\n")[:-l]
steps_union = "\n\n".join(steps) + "\n\n"

reward_context = [
    {"role":"user", "content":question},
    {"role":"assistant", "content":"<extra_0>".join(steps) + "<extra_0>"}
]

input_reward = reward_tokenizer.apply_chat_template(
    reward_context,
    add_generation_prompt=False,
    tokenize=False
)

# 采样思考
outputs: list[RequestOutput] = target_model.generate(
    input_prefix + steps_union,
    sampling_params=generating_params_1
)

next_steps = [item.text for item in outputs[0].outputs]

# 对采样的思考打分
rewards = reward_model.encode([input_reward + next_step + "<extra_0>" for next_step in next_steps])

# 取得分最高的 3 个思考
beam = [(reward.outputs.data.cpu().tolist()[-1][1], next_step) for reward, next_step in zip(rewards, next_steps)]
beam.sort(key=lambda x: x[0], reverse=True)
beam = beam[:3]

clear_output(wait=True)
print(f"User: {question}\n\nAssistant:\n{'\n--------------------------------------------------------------------------------\n'.join(['Sequence ' + str(j) + "\n\n" + steps_union + b[1] + '\n\nBeam Search: [' + str(b[0]) + ']' for j, b in enumerate(beam)])}")

# 将 k 个思考加入序列
inputs_target = [input_prefix + steps_union + step + "\n\n" for _, step in beam]

# Beam Search
while(flag and i < 10):

    beam_k_sqr = []

    # 对 k 个输入进行采样
    outputs: list[RequestOutput] = target_model.generate(
        inputs_target,
        sampling_params=sampling_params
    )

    for j, output in enumerate(outputs):
        next_steps = [item.text for item in output.outputs]

        # 对采样的思考打分
        rewards = reward_model.encode([input_reward + next_step + "<extra_0>" for next_step in next_steps])

        # 取得分最高的 3 个思考
        b = [(reward.outputs.data.cpu().tolist()[0][1], next_step) for reward, next_step in zip(rewards, next_steps)]
        b.sort(key=lambda x: x[0], reverse=True)
        b = b[:3]

        # 扩展概率
        for score, step in b:
            beam_k_sqr.append((score * beam[j][0], beam[j][1] + "\n\n" + step))

    beam_k_sqr.sort(key=lambda x: x[0], reverse=True)
    beam = beam_k_sqr[:3]

    if all(b[0] < 0.5 for b in beam):
        beam[:] = [(score * 2, text) for score, text in beam]

    clear_output(wait=True)
    print(f"User: {question}\nAssistant:\n{'\n--------------------------------------------------------------------------------\n'.join(['Sequence ' + str(j) + "\n\n" + steps_union + b[1] + '\n\nBeam Search: [' + str(b[0]) + ']' for j, b in enumerate(beam)])}")

    # 将 3 个思考加入序列
    inputs_target = [input_prefix + steps_union + text + "\n\n" for _, text in beam]

    flag = not all("</think>" in b[1] for b in beam)

    i += 1

# 第三阶段采样

results = []

for input_str in input_strs:
    output: list[RequestOutput] = target_model.generate(
        input_str,
        sampling_params=generating_params_2
    )
    results.append(output[0].outputs[0].text)# -------- Token 显示工具：将 "Ġcapital" 变为 " capital" --------
def pretty_token(tok: str):
    # GPT/Llama 字节级 BPE: Ġ 表示前导空格
    if tok.startswith("Ġ"):
        return " " + tok[1:]
    return tok

# ===== 2. 参数配置 =====
sampling_params = SamplingParams(
    max_tokens=1,        # 不生成，只分析 prompt
    temperature=1,
    prompt_logprobs=1   # ⭐ 只返回 top-1
)

tokenizer = llm.get_tokenizer()

import random
import json

def counterfactual_gsm8k(data: dict) -> str:
    ratios = [1/3, 0.5, 2, 3]
    question = data["question"]
    correct_answer = int(data["answer"].split('#### ')[-1])
    incorrect_answer = str(random.sample([int(correct_answer * ratio) for ratio in ratios], 1)[0])
    return f"{question} Why the answer is {tokenizer.pad_token}?"
    # return f"{question} Why {correct_answer} is incorrect?"
    # return f"{question} There is a trap in the question, you won't figure out the correct answer at once."


dataset = {"gsm8k": "/root/project/dos/dataset/ours/gsm8k_sample.jsonl"}

with open(dataset["gsm8k"], "r") as f:
    counterfactuals = [counterfactual_gsm8k(json.loads(line)) for line in f.readlines()]

print(f"[Counterfactual Question]:\n{counterfactuals[0]}")

prompt = tokenizer.apply_chat_template(
    [{'role':'user','content':counterfactuals[0]}],
    tokenize=False,
    add_generation_prompt=True
)

# ===== 3. 调用推理 =====
outputs: list[RequestOutput] = llm.generate(prompt, sampling_params)
req_out = outputs[0]

prompt_token_ids = req_out.prompt_token_ids            # 长度 = L
prompt_logprobs = req_out.prompt_logprobs              # 长度 = L - 1
prompt_tokens = tokenizer.convert_ids_to_tokens(prompt_token_ids)
# ===== 4. 打印表头 =====
print("Idx | Prompt_Token     | Decoded   | Top1_Token       | Top1_Decoded | LogProb     | Prob     | Match")
print("----+------------------+-----------+-------------------+--------------+-------------+----------+-------")

probs_plot = []


# ===== 5. 逐 token 对齐打印 =====
for i, (tok_id, tok_str) in enumerate(zip(prompt_token_ids, prompt_tokens)):

    decoded_tok = tokenizer.decode([tok_id])
    pretty_tok = pretty_token(tok_str)

    if i == 0:
        # 第一个 token 无 logprob、无 top-1
        print(f"{i:3d} | {pretty_tok:16s} | {decoded_tok:9s} | {'N/A':17s} | {'N/A':12s} | {'N/A':11s} | {'N/A':8s} |   -  ")
        probs_plot.append(float("nan"))
        continue

    # 第 i 个 token 的 logprob 来自 prompt_logprobs[i-1]
    step_dict = prompt_logprobs[i]   # dict[token_id → LogprobObj]

    # 取 top-1 的 token_id
    top1_id, lp_obj = max(step_dict.items(), key=lambda kv: kv[1].logprob)

    top1_tok_str = tokenizer.convert_ids_to_tokens([top1_id])[0]
    top1_decoded = tokenizer.decode([top1_id])

    logprob = lp_obj.logprob
    prob = math.exp(logprob)
    probs_plot.append(prob)

    # 比较 top-1 token 是否等于原本的 prompt token
    match = (top1_decoded == decoded_tok)

    print(
        f"{i:3d} | {pretty_tok:16s} | {decoded_tok:9s} | "
        f"{pretty_token(top1_tok_str):17s} | {top1_decoded:12s} | "
        f"{logprob:11.4f} | {prob:8.4f} | {'YES' if match else 'NO '}"
    )


# ===== 6. 绘图 =====
plt.figure(figsize=(10, 4))
plt.plot(range(len(probs_plot)), probs_plot, marker="o")
plt.xlabel("Token index")
plt.ylabel("Top-1 Probability of Actual Token")
plt.title("Prompt Token Probabilities (first token has no logprob)")
plt.grid(True)
plt.tight_layout()
plt.show()# 拿 tokenizer 解码 token，便于展示
tokenizer = llm.get_tokenizer()

# ===== 2. 配置 SamplingParams =====
sampling_params = SamplingParams(
    max_tokens=1,          # 这里只想算 prompt，不需要生成新 token
    temperature=1,       # 随便设，这里不采样
    prompt_logprobs=len(tokenizer)   # ⭐ 对每个 prompt token 返回全 vocab 的 logprobs
)

prompt = tokenizer.apply_chat_template(
    [{"role":"user","content":counterfactuals[0]}],
    add_generation_prompt=True,
    tokenize=False
)

# ===== 3. 调用 vLLM 推理 =====
outputs: list[RequestOutput] = llm.generate(prompt, sampling_params)
req_out = outputs[0]  # 只有一个 prompt，所以取第 0 个

prompt_token_ids = req_out.prompt_token_ids         # list[int]
prompt_logprobs = req_out.prompt_logprobs           # list[dict[token_id -> LogprobObj]]
prompt_tokens = tokenizer.convert_ids_to_tokens(prompt_token_ids)

# ===== 4. 对齐打印：每个 token 的 logprob & prob =====
print("Idx | Token                     |   LogProb    |    Prob")
print("----+---------------------------+-------------+-----------")

probs_for_plot = []

for i, (tok_id, tok_str) in enumerate(zip(prompt_token_ids, prompt_tokens)):
    # 第一个 token 没有对应的 logprob
    if i == 0:
        logprob = float("nan")
        prob = float("nan")
        continue

    # 第 i 个 token 的 logprob 来自 prompt_logprobs[i-1]
    step_dict = prompt_logprobs[i]  # dict[token_id -> LogprobObj]
    lp_obj = step_dict.get(tok_id)

    if lp_obj is None:
        # 说明该 token 不在返回的 top-K 里（如果你用的是 prompt_logprobs=K）
        logprob = float("nan")
        prob = float("nan")
    else:
        logprob = lp_obj.logprob
        prob = math.exp(logprob)

    probs_for_plot.append(prob)

    display_tok = tok_str.replace("\n", "\\n")
    if len(display_tok) > 25:
        display_tok = display_tok[:22] + "..."

    if math.isnan(logprob):
        lp_str = "   N/A"
        p_str = "   N/A"
    else:
        lp_str = f"{logprob:11.4f}"
        p_str = f"{prob:9.4f}"

    print(f"{i:3d} | {display_tok:25s} | {lp_str} | {p_str}")

# ===== 5. 用 matplotlib 可视化每个 token 的 prob =====
plt.figure(figsize=(10, 4))
plt.plot(range(len(probs_for_plot)), probs_for_plot, marker="o")
plt.xlabel("Token index in prompt")
plt.ylabel("P(actual token)")
plt.title("Prompt token probabilities")
plt.grid(True)

# 在 notebook 里直接 show；脚本环境可以改成 savefig
plt.tight_layout()
plt.show()
# 或者：
# plt.savefig("prompt_token_probs.png")